In [1]:
import pandas as pd
import numpy as np
import joblib


import json
from pandas.io.json import json_normalize

import requests
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus
from datetime import datetime,date,timedelta


import pymysql

## 기상청 API 날씨 데이터 불러오기

In [2]:
from datetime import datetime,date,timedelta
yesterday = date.today()-timedelta(1)
yesterday = yesterday.strftime('%Y%m%d')

# startday = yesterday
startday = '20221031'


url = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'

service_key = '***'

queryParams = '?' + urlencode({ quote_plus('ServiceKey') : service_key, 
                               quote_plus('pageNo') : '1',
                               quote_plus('numOfRows') : '700', 
                               quote_plus('dataType') : 'JSON', 
                               quote_plus('dataCd') : 'ASOS', 
                               quote_plus('dateCd') : 'HR', 
                               quote_plus('startDt') : startday,
                               quote_plus('startHh') : '00', 
                               quote_plus('endDt') :  yesterday,
                               quote_plus('endHh') : '23', 
                               quote_plus('stnIds') : '108' })

In [3]:
# print(datetime.now())

In [3]:
url + queryParams

'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList?ServiceKey=1rTRcPcrgRX4bckCMPyIIgsMgxqzwZwAKsqCjJe74xrEwdc2rQRZgHAZ60aJdhT6313RnB8znsO2jJONz%2Bltow%3D%3D&pageNo=1&numOfRows=700&dataType=JSON&dataCd=ASOS&dateCd=HR&startDt=20221031&startHh=00&endDt=20221102&endHh=23&stnIds=108'

In [4]:
res = requests.get(url + queryParams)
documents=json.loads(res.text)

weather = pd.json_normalize(documents['response']['body']['items']['item'])
weather

,tm,rnum,stnId,stnNm,ta,taQcflg,rn,rnQcflg,ws,wsQcflg,...,lcsCh,vs,gndSttCd,dmstMtphNo,ts,tsQcflg,m005Te,m01Te,m02Te,m03Te
0,2022-10-31 00:00,1,108,서울,12.4,,,,1.8,,...,,2000,,,9.7,,14.5,14.6,15.0,15.6
1,2022-10-31 01:00,2,108,서울,12.6,,,,2.1,,...,42,2000,,,10.2,,14.3,14.4,14.9,15.6
2,2022-10-31 02:00,3,108,서울,12.4,,,,2.4,,...,42,2000,,,10.1,,14.2,14.3,14.8,15.5
3,2022-10-31 03:00,4,108,서울,12.2,,,,1.6,,...,35,2000,,,10.1,,14.0,14.1,14.6,15.5
4,2022-10-31 04:00,5,108,서울,12.2,,,,2.0,,...,33,2000,,,10.4,,13.9,14.0,14.5,15.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,2022-11-02 19:00,68,108,서울,11.9,,,,2.3,,...,,2000,,,9.5,,14.3,14.2,14.2,14.6
68,2022-11-02 20:00,69,108,서울,11.2,,,,2.6,,...,60,2000,,,8.7,,13.9,14.0,14.2,14.7
69,2022-11-02 21:00,70,108,서울,11.0,,,,2.5,,...,,2000,,,8.3,,13.6,13.8,14.2,14.8
70,2022-11-02 22:00,71,108,서울,10.4,,,,1.7,,...,55,2000,,,7.6,,13.3,13.5,14.1,14.8


In [5]:
col = ['tm','stnId','stnNm','ta','rn','ws','hm','dsnw']

weather=weather[col]
weather.columns = ['일시','지점','지점명','기온','강수량','풍속','습도','적설량']
weather

,일시,지점,지점명,기온,강수량,풍속,습도,적설량
0,2022-10-31 00:00,108,서울,12.4,,1.8,70,
1,2022-10-31 01:00,108,서울,12.6,,2.1,71,
2,2022-10-31 02:00,108,서울,12.4,,2.4,72,
3,2022-10-31 03:00,108,서울,12.2,,1.6,74,
4,2022-10-31 04:00,108,서울,12.2,,2.0,75,
...,...,...,...,...,...,...,...,...
67,2022-11-02 19:00,108,서울,11.9,,2.3,53,
68,2022-11-02 20:00,108,서울,11.2,,2.6,58,
69,2022-11-02 21:00,108,서울,11.0,,2.5,64,
70,2022-11-02 22:00,108,서울,10.4,,1.7,65,


In [6]:
weather.loc[weather['강수량'] == '', '강수량'] = 0
weather.loc[weather['적설량'] == '', '적설량'] = 0
weather['일시'] = pd.to_datetime(weather['일시'])
weather['지점'] = weather['지점'].astype('int64')
weather['기온'] = weather['기온'].astype('float')
weather['강수량'] = weather['강수량'].astype('float')
weather['풍속'] = weather['풍속'].astype('float')
weather['적설량'] = weather['적설량'].astype('float')
weather['지점명'] = '서울특별시'
weather

<ipython-input-6-b8cfa2e75664>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather.loc[weather['강수량'] == '', '강수량'] = 0
<ipython-input-6-b8cfa2e75664>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather.loc[weather['적설량'] == '', '적설량'] = 0
<ipython-input-6-b8cfa2e75664>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather['일시'] = pd.to_datetime(weather['일시'])
<ipyth

,일시,지점,지점명,기온,강수량,풍속,습도,적설량
0,2022-10-31 00:00:00,108,서울특별시,12.4,0.0,1.8,70,0.0
1,2022-10-31 01:00:00,108,서울특별시,12.6,0.0,2.1,71,0.0
2,2022-10-31 02:00:00,108,서울특별시,12.4,0.0,2.4,72,0.0
3,2022-10-31 03:00:00,108,서울특별시,12.2,0.0,1.6,74,0.0
4,2022-10-31 04:00:00,108,서울특별시,12.2,0.0,2.0,75,0.0
...,...,...,...,...,...,...,...,...
67,2022-11-02 19:00:00,108,서울특별시,11.9,0.0,2.3,53,0.0
68,2022-11-02 20:00:00,108,서울특별시,11.2,0.0,2.6,58,0.0
69,2022-11-02 21:00:00,108,서울특별시,11.0,0.0,2.5,64,0.0
70,2022-11-02 22:00:00,108,서울특별시,10.4,0.0,1.7,65,0.0


## AWS 다운로드

In [7]:
import pandas as pd
import numpy as np
import joblib


# import json
# from pandas.io.json import json_normalize

# import requests
# from bs4 import BeautifulSoup
# from urllib.request import Request, urlopen
# from urllib.parse import urlencode, quote_plus
from datetime import datetime,date,timedelta

import pymysql

In [7]:
w_df = pd.read_csv("../../dataset/global/OBS_AWS_TIM_20220408104453.csv",encoding='cp949')
# w_df.columns = ['일시','지점','지점명','기온','강수량','풍속','습도']
col = ['일시','지점','지점명','기온(°C)','강수량(mm)','풍속(m/s)','습도(%)']

w_df = w_df[col]
w_df['적설량'] = 0
w_df.columns = ['일시','지점','지점명','기온','강수량','풍속','습도','적설량']
w_df['일시']  = pd.to_datetime(w_df['일시'])
w_df

,일시,지점,지점명,기온,강수량,풍속,습도,적설량
0,2022-04-07 00:00:00,116,관악(레),7.4,0.0,2.9,28.2,0
1,2022-04-07 01:00:00,116,관악(레),6.6,0.0,3.8,47.3,0
2,2022-04-07 02:00:00,116,관악(레),5.7,0.0,4.2,51.4,0
3,2022-04-07 03:00:00,116,관악(레),5.3,0.0,4.2,49.0,0
4,2022-04-07 04:00:00,116,관악(레),4.7,0.0,3.5,52.5,0
...,...,...,...,...,...,...,...,...
667,2022-04-07 19:00:00,889,현충원,10.7,0.0,3.2,29.9,0
668,2022-04-07 20:00:00,889,현충원,9.7,0.0,2.1,46.6,0
669,2022-04-07 21:00:00,889,현충원,9.2,0.0,0.9,51.7,0
670,2022-04-07 22:00:00,889,현충원,8.6,0.0,1.4,59.7,0


In [7]:
w_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   일시      672 non-null    datetime64[ns]
 1   지점      672 non-null    int64         
 2   지점명     672 non-null    object        
 3   기온      672 non-null    float64       
 4   강수량     671 non-null    float64       
 5   풍속      672 non-null    float64       
 6   습도      528 non-null    float64       
 7   적설량     672 non-null    int64         
dtypes: datetime64[ns](1), float64(4), int64(2), object(1)
memory usage: 42.1+ KB


In [8]:
weather = w_df.groupby('일시').mean().reset_index()
weather['지점'] = 108
weather['지점명'] = '서울특별시'

f_col = ['일시','지점','지점명','기온','강수량','풍속','습도','적설량']

weather = weather[f_col]
weather

,일시,지점,지점명,기온,강수량,풍속,습도,적설량
0,2022-04-07 00:00:00,108,서울특별시,10.635714,0.0,1.175000,39.700000,0.0
1,2022-04-07 01:00:00,108,서울특별시,10.835714,0.0,1.442857,41.450000,0.0
2,2022-04-07 02:00:00,108,서울특별시,10.617857,0.0,1.825000,39.890909,0.0
3,2022-04-07 03:00:00,108,서울특별시,10.175000,0.0,2.000000,40.213636,0.0
4,2022-04-07 04:00:00,108,서울특별시,9.850000,0.0,1.360714,41.622727,0.0
5,2022-04-07 05:00:00,108,서울특별시,9.432143,0.0,1.307143,45.354545,0.0
6,2022-04-07 06:00:00,108,서울특별시,9.196429,0.0,1.200000,48.218182,0.0
7,2022-04-07 07:00:00,108,서울특별시,9.150000,0.0,0.975000,49.454545,0.0
8,2022-04-07 08:00:00,108,서울특별시,9.789286,0.0,1.439286,47.477273,0.0
9,2022-04-07 09:00:00,108,서울특별시,11.203571,0.0,1.582143,41.031818,0.0


## DB 추가

In [7]:
host = '****'
username = '***'
password = '***'
port = "****"
database = '*****'

conn = pymysql.connect(host=host,user=username,db=database, port=port, password=password)
curs=conn.cursor(pymysql.cursors.DictCursor)

In [8]:
query = """INSERT INTO `pos`.`im_service_weather_info`(local_id, local_name, temperature, rainfall, wind_speed, humidity,snow, reg_datetime)
           values (%s,%s,%s,%s,%s,%s,%s,%s)"""


for i in range(len(weather)):
    
    local_id = weather['지점'][i]
    local_name = weather['지점명'][i]
    temp = weather['기온'][i]
    rain = weather['강수량'][i]
    wind = weather['풍속'][i]
    humidity = weather['습도'][i]
    snow = weather['적설량'][i]
    reg_date = weather['일시'][i]
        
    curs.execute(query, (local_id, local_name,temp ,rain, wind, humidity,snow, reg_date))

    conn.commit()
    
curs.close()
conn.close()